<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Visualization

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
from ipywidgets import interact
#setting plot size
plt.rcParams["figure.figsize"] = (7.0,4.0)

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['2022_10_14_script.xlsx']))

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Creating time categories
df1 = df1.assign(year = df1['order_date'].dt.year,
                 quarter = df1['order_date'].dt.quarter,
                 month = df1['order_date'].dt.month,
                 month_name = df1['order_date'].dt.month_name(),
                 weeknum_order = df1['order_date'].dt.isocalendar().week,
                 weeknum_activation = df1['activation_date'].dt.isocalendar().week,
                 weeknum_cancellation = df1['cancellation_date'].dt.isocalendar().week,
                 day = df1['order_date'].dt.day_name()
                 )

In [ ]:
# Checking the dataframe info
df1.info()

In [ ]:
# Splitting columns to lists for interactive visualization
date_col = df1.select_dtypes(include=['datetime64[ns]']).columns.tolist()
dim_col = df1.select_dtypes(include=['object']).columns.tolist()
cat_col = [i for i in dim_col if ('category') in i]
num_col = df1.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [ ]:
num_col

In [ ]:
df1[dim_col[0]].unique()

In [ ]:
df1[(df1['product_type']=='100 GB')&(df1['usage_category']=='Has_Usage')].drop(columns=['rgu_count'], axis=1)

In [ ]:
num_col[1]

## Visualization

In [ ]:
# Exploring numeric columns by Category
@interact(Period=date_col, Dimension=df1[dim_col[0]].unique(), Category=cat_col, Measure=num_col)
def visualize_category(Period, Dimension, Category, Measure):
  df = data=df1[(df1[dim_col[0]]==Dimension)].copy()
  sns.lineplot(x=Period, y=Measure, data=df, hue=Category)
  #plt.title(f'Product_type: {Dimension}; Measure: {Measure}; Category: {Category}')
  plt.title(f'Product_type: {Dimension}; Measure: {Measure}')
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

In [ ]:
# Exploring numeric columns
@interact(Period=date_col, Dimension=df1[dim_col[0]].unique(), Category=cat_col, Measure=num_col)
def visualize_statistics(Period, Dimension, Category):
  df = data=df1[(df1[dim_col[0]]==Dimension)&(df1[cat_col[0]]=='Has_Usage')].copy()
  sns.lineplot(x=Period, y=num_col[2], data=df, color='red')
  #sns.lineplot(x=Period, y=num_col[3], data=df)
  #sns.lineplot(x=Period, y=num_col[4], data=df)
  sns.lineplot(x=Period, y=num_col[5], data=df, color='black')
  plt.title(f'Product_type: {Dimension}; Category: Has_Usage')
  plt.legend(labels=[num_col[1], num_col[4]], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()